In [1]:
import numpy as np
import json
import cherrypy
import tensorflow as tf

In [ ]:
#code for using a tflite model for prediction


interpreter = tf.lite.Interpreter(model_path=models_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()



# input_data = np.array([temperature_list, humidity_list], dtype=np.float32)
# input_data = input_data.reshape(1, 6, 2)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
predict_result = interpreter.get_tensor(output_details[0]['index'])

